# Lab#4 - LTE Handover Analysis with Wireshark
***
# 1. Introduction:
***

In this exercise you will analyze how S1 and X2 Handover procedures are performed in LTE. You will be given two pcap files containing packet captures of the procedures. Your task will be to analyze the packet captures and relate them to the theory presented in the lectures. You will also be provided with a discussion of the procedures that serves as an introduction to each task.

***
# 2. Introduction to the capture files
***

The __X2Handover.pcap__ is a large packet capture containing different information. One of the challenges when dealing with such files is to apply the correct filters, in order to reduce the number of packets to be analysed. By studying [picture 1](#X2_Handover) below, we can identify the necessary interfaces we need to study.

> **NB: RRC signaling is not present** 

The __S1Handover.pcap__ is a concise packet capture done on the S1 interface during a handover between two eNodeBs. The tasks will involve analysing the sequence of the procedure step-by-step.

> **NB 1: only S1 interface capture is present.**

*** 
# 3. X2 Handover: the procedure
***

<a id='X2_Handover'></a>
<center>
    <figure>
        <img src= "Figures/X2Handover.png" alt="first" class="bg-primary mb-1" width="1500px">
        <center><figcaption><b>1. X2 Handover procedure (reference <a href="https://www.3glteinfo.com/intra-lte-handover-using-x2-interface/">here</a>).</b></figcaption></center>
    </figure>
</center>


<!-- To create a link to an image: [Sentence for the link](#id_of_pic) -->
Handover is a procedure where the User Equipment (UE) changes its serving eNodeB to a neighbouring eNodeB based on signal measurments. An X2 Handover is executed for a certain UE when at least two factors occur:
- the UE receives stronger measurments from a neighbouring eNode-B than the serving eNode-B;
- there is an existing X2 interface between the two eNodeBs. 

The X2 interface ensures direct communication between two eNodeBs and for this reason this procedure is considered the most efficient handover type.    

X2 handover can be divided into three steps:
- __Handover Preparation__: in function of the uplink and downlinks measurements, the S-eNodeB decides to handover to T-eNodeB. The S-enodeB asks to the T-eNodeB whether it has enough resources to cover a new UE. If so, the T-eNodeB replies to the S-eNodeB with an acknowledgement (note that the Evolved Packet Core (EPC) is not involved); 
- __Handover Execution__: the S-eNodeB allows the UE to perform the Random Access Procedure towards the T-eNodeB. At the end of this step, all the downlink packets passes through the S-eNodeB to the T-eNodeB, istead the uplink packets (when the Random Access Procedure is completed) are diectly forwarded to the T-eNodeB (see [picture 7](#UL_DL_bearers_before_HO));
- __Handover Completion__: in this part, the MME says to the Serving-GW to no longer forwards the downlink packets towards the S-eNodeB, but towards the new one (S1 bearer change phase), in order to obtain the usual EPS bearer as shown in [picture 10](#UL_DL_bearers_after_HO). Furthermore, the S-eNodeB releases the resources previously assigned to the UE.

Use the pictures as shown below, in order to better visualise protocols and entities involved.


<a id='UsefulPic'></a>
<center>
    <figure>
        <img src= "Figures/UsefulPic.PNG" alt="first" class="bg-primary mb-1" width="1100px">
    </figure>
</center>

***
## 3.1. Handover Preparation

<a id='X2HandoverPreparation'></a>
<center>
    <figure>
        <img src= "Figures/X2HandoverPreparation.PNG" alt="first" class="bg-primary mb-1" width="400px">
        <figcaption><b>5. X2 Handover Preparation procedure.</b></figcaption>
    </figure>
</center>

- <u>Measurement Report</u>: air interface (RRC protocol) involved. The S-eNodeB sends the Measurement Report Request message Its purpose is to instruct the UE to send a measurement report to the network as soon as it detects the thresholds. The UE sends all the required measurements regarding the Serving Cell (i.e. the camping cell where the Serving eNodeB (S-eNodeB) is present) and for all the Neighbor Cells (i.e. the cells where the signal perceived by the UE is strong enough), like RSRP (Reference Signal Received Power) and RSRQ (Reference Signal Received Quality). By exploiting the information mentioned before (i.e. downlink measurements) and further S-eNodeB information (i.e. uplink measurements), it makes the decision to carry on the handover to a Target eNodeB (T-eNodeB) using the handover algorithm; 

> **NB**: each network operator could have its own handover algorithm.

- <u>Handover Request</u>: X2 interface (X2AP protocol) involved. The S-eNodeB sends it to the T-eNodeB passing the necessary information to prepare the handover at the target side (e.g., the Target Cell Id, the GUMMEI (Globally Unique MME Identifier) and the eRABs to be setup).

- <u>Handover Request Ack</u>: X2 interface (X2AP protocol) involved. The T-eNodeB checks for resource availability and, if available, reserves the resources and sends back the acknowledgement message, including a transparent container to be sent to the UE as an RRC message. The container includes a new C-RNTI (Cell Radio Network Temporary Identifier), T-eNodeB security algorithm identifiers for the selected security algorithms, and may include a dedicated RACH preamble and possibly some other parameters. At this point, the X2 Transport Bearer for the UE downlink data is established, and used to forward the packets in the User Plane from S-eNodeB to T-eNodeB.

-------
### **3.1.1. Interactive session:**
-------

Open the __X2Handover.pcap__ file. In order to show only X2 interface packets, we have to filter the corresponding protocol, such as X2AP. The command to by used is shown hereafter:

__`x2ap`__

You must see something like as shown in the following picture:

<center>
    <figure>
        <img src= "Figures/HO_request_and_ack.PNG" alt="first" class="bg-primary mb-1" width="400px">
    </figure>
</center>

#### <u>Question 3.1.1a</u>:

Identify the packet number of *both Handover Request* and *Handover Request Ack*. Write the answers in the following code block. Furthermore, write down the IP address of the T-eNodeB and the S-eNodeB.

In [1]:
HO_Request_and_ACK = {    
    "HO Request pkt num": ,
    "HO Request Ack pkt num": ,
    "S-eNodeB": ,
    "T-eNodeB": ,
}

***
## 3.2. Handover Execution

<a id='X2HandoverExecution'></a>
<center>
    <figure>
        <img src= "Figures/X2HandoverExecution.PNG" alt="first" class="bg-primary mb-1" width="900px">
        <figcaption><b>6. X2 Handover Execution procedure.</b></figcaption>
    </figure>
</center>

- <u>RRC Connection Reconfiguration</u>: air interface (RRC protocol) involved. The S-eNodeB sends it to the UE to perform the handover (you can find *handoverType* as a field of this packet, which defines the type of handover). The S-eNodeB performs the necessary integrity protection and ciphering of the message and sends it to the UE.

- <u>SN (Sequence Number) Status Transfer</u>: X2 interface (X2AP protocol) involved. The S-eNodeB sends the eNodeB STATUS TRANSFER message to the T-eNodeB to convey the PDCP (both UL and DL) of the eRABs. After this, the S-eNodeB stops sending the DL data to the UE, and starts sending it to the T-eNodeB through the X2 interface. The T-eNodeB has to keep in the buffer the downlink data since the UE hasn't completed the Random Access Procedure yet.

- <u>Random Access Procedure</u>: air interface (RRC protocol) involved. The random access procedure is carried out with the T-eNodeB. There are two types of random access procedure: non-contention (or contentio-free) and contention (discussed during the __Attach procedure__). In the first one, present in both the handover types, one of the 64 preambles is assigned to the UE. Hereafter, the two sequences are shown.

<a id='Cont_and_NONCont_RACH'></a>
<center>
    <figure>
        <img src= "Figures/Cont_and_NONCont_RACH.PNG" alt="first" class="bg-primary mb-1" width="600px">
        <figcaption><b>7. Different signaling between contention and contention-free random access procedure.</b></figcaption>
    </figure>
</center>

**UL/DL data path after Handover Execution**: The S-eNodeB starts forwarding the downlink data packets to the T-eNodeB through the X2 interface for all the data bearers (which are being established in the T-eNodeB during the *Handover Request* message processing), and in the meanwhile, the UE sends to the T-eNodeB the uplink data (since the Random Access Procedure is performed). In [picture 7](#UL_DL_bearers_before_HO) are shown the beares involved.

<a id='UL_DL_bearers_before_HO'></a>
<center>
    <figure>
        <img src= "Figures/UL_DL_bearers_before_HO.PNG" alt="first" class="bg-primary mb-1" width="900px">
        <figcaption><b>8. DL UE data forwarding to T-eNodeB and new UL UE data path: bearers involved (reference <a href="https://www.3glteinfo.com/intra-lte-handover-using-x2-interface/">here</a>).</b></figcaption>
    </figure>
</center>

-------
### **3.2.1. Interactive session:**
-------

Return to the __X2Handover.pcap__ file. 

#### <u>Question 3.2.1a</u>:

Take a look to [picture 1](#X2_Handover) and [7](#UL_DL_bearers_before_HO). At this point, the UE is served by the T-eNodeB. Why can uplink packets be delivered by the T-eNodeB to the Serving-GW, but the downlink packets must be delivered by the Serving-GW to the S-eNodeB?

> **Hint**:  think about the messages exchanged until this point. What does Serving-GW know about the current handover procedure?

#### <u>Question 3.2.1b</u>:

From the point of view of the UE, how can you categorize the handover in LTE?  

#### <u>Question 3.2.1c</u>:

As you know, in UMTS there are two types of handover: soft and hard. Explain why in LTE only one type of handover can be managed, and why in UMTS both types are available.

> **Hint**: think about which medium access protocols are involved. Furthermore, consider why orthogonality between the radio channels is so important.

#### <u>Question 3.2.1d</u>:

Considering [picture 7](#UL_DL_bearers_before_HO), which is the main advantage of usign contention-free based random access instead of the normal one?

---
## 3.3. Handover Completion

<a id='X2HandoverCompletion'></a>
<center>
    <figure>
        <img src= "Figures/X2HandoverCompletion.PNG" alt="first" class="bg-primary mb-1" width="900px">
        <figcaption><b>9. X2 Handover Completion procedure.</b></figcaption>
    </figure>
</center>

- <u>Path Switch Request</u>: S1 interface (S1AP protocol) involved. The purpose of the *Path Switch Request* message is to request the switch of a downlink GTP tunnel towards a new GTP tunnel endpoint (i.e. change the old S1 tunnel to the new one). The T-eNodeB informs the MME that the UE has changed cell, including the TAI (Tracking Area Identity) and ECGI (E-UTRAN Cell Global Identifier) of the target. The MME determines that the Serving-GW can continue to serve the UE.

- <u>Modify Bearer Request</u>: already explained in the previous ***Attach Procedure lab excercise***.

- <u>Modify Bearer Request Ack</u>: already explained in the previous ***Attach Procedure lab exercise***. After this, the Serving-GW sends one or more End Marker packets on the old path to the S-eNodeB and then can release any user plane resources toward the S-eNodeB. You can see the path of the EM packet in [picture 9](#UL_DL_bearers_before_HO).

<a id='ME'></a>
<center>
    <figure>
        <img src= "Figures/EM.PNG" alt="first" class="bg-primary mb-1" width="900px">
        <figcaption><b>10. EM packet path, in order to release the old path (reference <a href="https://www.3glteinfo.com/intra-lte-handover-using-x2-interface/">here</a>).</b></figcaption>
    </figure>
</center>

- <u>Path Switch Request Ack</u>: S1 interface (S1AP protocol) involved. The MME responds to the T-eNodeB notifying the completion of the handover.

- <u>Release Resource</u>: X2 interface (X2AP protocol) involved. Called *UE Context Release* as well, it's sent by the T-eNodeB to the S-eNodeB: it communicates that user and control plane resources for the associated UE context are allowed to be released.

**UL/DL data path after Handover Completion**: see EPS bearer is correctly created, considering the target eNodeB.


<a id='UL_DL_bearers_after_HO'></a>
<center>
    <figure>
        <img src= "Figures/UL_DL_bearers_after_HO.PNG" alt="first" class="bg-primary mb-1" width="900px">
        <figcaption><b>11. EPS bearer/s established after Handover Completion (reference <a href="https://www.3glteinfo.com/intra-lte-handover-using-x2-interface/">here</a>).</b></figcaption>
    </figure>
</center>

-------
### **3.3.1. Interactive session:**
-------

Open the __X2Handover.pcap__ file. Use again the filter command to visualize just X2AP packets.

#### <u>Question 3.3.1a</u>:

Write down the *Release Resource* packet number. Write down the answer in the following code block.

In [2]:
ReleaseResource = {
    "packet numer" : 
}

-------

*** 
# 4. S1 Handover: the procedure
***

<a id='S1Handover'></a>
<center>
    <figure>
        <img src= "Figures/S1Handover.PNG" alt="first" class="bg-primary mb-1" width="700px">
        <figcaption><b>12. S1 Handover procedure.</b></figcaption>
    </figure>
</center>

In a S1 Handover there is no direct connection (X2 interface) between the involved eNBs. Hence, the MME needs to be involved in the communication between the two entities. This communication is done over the S1 interface and is thus given its name, S1 Handover. In general, a S1 Handover is executed only when there is no present X2 interface. Note that the UE does not experience any difference in whether an X2 or S1 Handover is performed. 

-------
### **4. Interactive session:**
-------

#### <u>Question 4a</u>:
Explain the main differences between the S1 and X2 Handovers, and in particular:
- Which is the faster handover? Why?
- Which is the difference between intra-RAT and inter-RAT handover? Which types do S1 and X2 handovers support?

#### <u>Question 4b</u>:
Describe how the UE is moving through cell sectors. Where does it end up in comparison to the starting point? Take into account the last question.

#### <u>Question 4c</u>:
Can any other LTE procedures be observed in the S1Handover.pcap file?

***
## 4.1. Handover Preperation

<a id='S1HO_preparation'></a>
<center>
    <figure>
        <img src= "Figures/S1HO_preparation.PNG" alt="first" class="bg-primary mb-1" width="700px">
        <figcaption><b>13. S1 Handover Preparation procedure.</b></figcaption>
    </figure>
</center>

- <u>Measurement Report</u>: see description on __X2 Handover__ notebook.

- <u>1) Handover Request</u>: S1 interface (S1AP protocol) involved. Since there is no X2 interface among S-eNodeB and T-eNodeB (so that, direct forwarding is not an option), the source eNodeB decides to send the Handover Request to the MME. Depending on the TAI (Tracking Area ID) of the T-eNodeB, it's possible that the MME doesn't cover that base station: in this case the MME forwards the notification towards the corresponding MME covering the eNodeB. For the sake of simplicity, in our case the MME discovers that the T-eNodeB is covered. You can find target eNodeB Identity, the target TAI and the reason that triggered the handover procedure. Finally, you can find a transparent container in which there are some radio datas about the source cell.

- <u>2) Handover Request</u>: S1 interface (S1AP protocol) involved. The MME forwards the Handover Request (generated by the S-eNodeB) towards the T-eNodeB. Hence, similar information are present. Furthermore, you can find the maximum aggregated bit-rate and a transparent container including the UE Capabilities, RRC Configuration, etc...

- <u>Handover Request Ack</u>: S1 interface (S1AP protocol) involved. The T-eNodeB allows the handover procedure to keep on going. You can find an EPS Bearer Setup List, that contains a list of addresses and TEIDs (one TEID per bearer) for downlink traffic between Serving GW and T-eNodeB (S1-U reference point): one DL S1 TEID that the T-eNodeB allocated to the Serving GW, and one DL S1 TEID that is referred to the indirect tunnel to reach the T-eNodeB. Furthermore, a transparent container is present, where the Handover Command message is carried. 

- <u>Create Indirect Forward Tunnel Request and Ack</u>: S11 interface (GTPv2 protocol) involved. The MME aks the Serving GW to create a temporary tunnel between S-eNodeB and T-eNodeB. When the handover procedure will be completed, the tunnel must be eliminated. As a consequence, the Serving GW confirms the creation of the tunnel, and the S-eNodeB can forward downlink data towards the new base station.

- <u>Handover Command</u>: S1 interface (S1AP protocol) involved. The MME informs S-eNodeB that the resources for the handover have been prepared at the target side (T-eNodeB). You can find the transparent container, where you can find radio related information through the EPC (Evolved Packet Core). You can find the dedicated resources to the UE in order to carry out the contention-free based random access procedure.


-------
### **4.1.1. Interactive session:**
-------

Open __S1Handover.pcap__. After this, you can use a filter to display only S1AP packets:

__`s1ap`__

In this task, you must discover the handover execution sequence. Use the S1Handover.pcap file and navigate to a similar view as shown hereafter:
<center>
    <figure>
        <img src= "Figures/HO_preparation_pcap.PNG" alt="first" class="bg-primary mb-1" width="600px">
    </figure>
</center>

#### <u>Question 4.1.1a</u>:

Write down the IP of S-eNodeB, T-eNodeB and MME of the first handover.

In [3]:
S1Interface_entities = {
    "S-eNodeB IP" : ,
    "MME IP" : ,
    "T-eNodeB IP" :  
}

#### <u>Question 4.1.1b</u>:

Write down the maximum bit-rate both for downlink and uplink traffic.

In [4]:
uEaggregateMaximumBitRate = {
    "uEaggregateMaximumBitRateDL" : , #bit/s
    "uEaggregateMaximumBitRateUL" :   #bit/s
}

#### <u>Question 4.1.1c</u>:

Write down the index of the assigned preamble to the UE. Furthermore, report the pcaket number where you find it.

> **Hint**: as already discussed, the transparent container is used to delivery radio related information through the EPC. Where can you find it?

In [5]:
preamble_id = {
    "ra-PreambleIndex" : ,
    "packet number" : 
}

#### <u>Question 4.1.1d</u>:

Why can we not see the creation of the Indirect Forward Tunnel between the MME and the Serving-GW? What inteface is the captured file monitored on?

#### <u>Question 4.1.1e</u>:

Write down the Cell-ID of the Target eNB and the pLMN identity for the first handover. In which packet can you first observe the Cell-ID (write down the packet number)?

In [6]:
Cell_ID = {
    "cell-id" : ,
    "plmn-id" : , #MNC and MCC are unknown, so I didn't put this question
    "packet numer" : 
}

#### <u>Question 4.1.1f</u>:

What is the Tracking Area Identity (TAI) comprised of and what is the importance of it in relation to handovers? Is it possible to identify the TAI in the packet capture?
If so, identify the elements comprising the TAI, write down the values and where you found them.


***
## 4.2. Handover execution

<a id='S1HO_execution'></a>
<center>
    <figure>
        <img src= "Figures/S1HO_execution.PNG" alt="first" class="bg-primary mb-1" width="800px">
        <figcaption><b>14. S1 Handover Execution procedure.</b></figcaption>
    </figure>
</center>

- <u>RRC Connection Reconfiguration</u>: see description on __X2 Handover__.

- <u>eNodeB Status Transfer</u>: S1 interface (S1AP protocol) involved. The message is generated by the S-eNodeB and sent to the T-eNodeB, passing through the MME. It includes (for each E-RABs) DL Count and UL Count, where the first one represents the count of the first packet to send to the UE, and the second one to receive from the UE. A count is a 32-bit value consisting of Hyper Frame Number (HFN) and PDCP Sequence Number (SN).

- <u>Random Access Procedure</u>: see description on __X2 Handover__.

- **UL/DL data path**: hereafter, you can see the user plane since this point.

<a id='UL_DL_traffic_flow_temp_tunnel_S1_HO'></a>
<center>
    <figure>
        <img src= "Figures/UL_DL_traffic_flow_temp_tunnel_S1_HO.PNG" alt="first" class="bg-primary mb-1" width="900px">
        <figcaption><b>10. EPS bearer/s established after Handover Completion.</b></figcaption>
    </figure>
</center>

-------
### **4.2.1. Interactive session:**
-------

In this task, you must discover the handover execution sequence. Use the S1Handover.pcap file and navigate to a similar view as shown hereafter:

<center>
    <figure>
        <img src= "Figures/HO_execution_pcap.PNG" alt="first" class="bg-primary mb-1" width="600px">
    </figure>
</center>

#### <u>Question 4.2.1a</u>:

Write down the uplink and downlink count values (where ***pDCD-SN*** and ***hFN*** stand for PDCP SN and HFN respectively). 

In [7]:
ul_COUNTvalue = {
    "pDCD-SN" : ,
    "hFN" : 
}

dl_COUNTvalue = {
    "pDCD-SN" : ,
    "hFN" : 
}

#### <u>Question 4.2.1b</u>:

Is it possible the eNodeB sends an uplink with PCDC SN equal to 9? 

#### <u>Question 4.2.1c</u>:

Why is it necessary to transfer the packet counter from the S-eNodeB to the T-eNodeB? Furthermore, explain how it is performed in an X2 handover. 

---
## 4.3. Handover Completion

<a id='S1HO_completion'></a>
<center>
    <figure>
        <img src= "Figures/S1HO_completion.PNG" alt="first" class="bg-primary mb-1" width="800px">
        <figcaption><b>15. S1 Handover Completion procedure.</b></figcaption>
    </figure>
</center>

- <u>Handover Notify</u>: S1 interface (S1AP protocol) involved. After the UE has carried out the random access procedure, the T-eNodeB informs the EPC (i.e. the MME) that the S1 handover is finished. The message includes the ECGI (E-UTRAN Cell Global Identifier) and TAI.

- <u>Modify Bearer Request and Response</u>: see description on __Attach Procedure__.

- <u>UE Context Release and Ack</u>: S1 interface (S1AP protocol) involved. MME informs S-eNodeB to release the resources (S1 bearer and indirect tunnel. See [picture ?](#UL_DL_traffic_flow_temp_tunnel_S1_HO), the S1 Bearer for Indirect Forwarding (UL)) used in the S1 interface and UE Context. Hence, S-eNodeB releases the resources and informs the MME by sending the Ack.

- <u>Deletion Indirect Tunnel and Response</u>: S11 interface (GTPv2 protocol) involved. The MME asks the Serving GW to release the indirect tunnel (see [picture ?](#UL_DL_traffic_flow_temp_tunnel_S1_HO), the S1 Bearer for Indirect Forwarding (DL)), and finally it replies with Ack after releasing the resources.

-------
### **4.3.1. Interactive session:**
-------

In this task, you must discover the handover execution sequence. Use the S1Handover.pcap file and navigate to a similar view as shown hereafter:
<center>
    <figure>
        <img src= "Figures/HO_completion_pcap.PNG" alt="first" class="bg-primary mb-1" width="600px">
    </figure>
</center>

#### <u>Question 4.3.1a</u>:

Write down the packet number of each of the displayed packets for the first handover.

In [8]:
S1_HO_Completion = {
    "Handover Notify" : ,
    "UE Context Release Command" : ,
    "UE Context Release Command Ack" : 
}

-------